In [1]:
import os
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.kdf.hkdf import HKDF

ModuleNotFoundError: No module named 'cryptography'

In [167]:
class KEM_RSA:
    def __init__(self, salt):
        self.private_key = None
        self.public_key  = None
        self.salt = salt

    
    # Metodos Privados
    def __encrypt(self, message, peer_public_key):
        n, e = peer_public_key
        # message ^ e % n
        return power_mod(message, e, n)

    def __decrypt(self, cipher_message):
        n, d = self.private_key
        # cipher_message ^ d % n
        return power_mod(cipher_message, d, n)


    # Metodos publicos
    def generate_keys(self, tam):
        # Dois primos q e p (tam deve ser no minimo 1024)
        q = random_prime(2 ^ (tam / 2) - 1, true, 2 ^ (tam / 2 - 1))
        p = random_prime(2 ^ (tam / 2) - 1, true, 2 ^ (tam / 2 - 1))
        
        # n é o produto de q e p
        n = p * q
        
        # Função do totiente de euler
        phi = (p - 1) * (q - 1)
        
        # Escolher um inteiro e tal que 1 < e < phi
        # e e phi têm de ser relativamente primos entre si
        # Para isso verificamos o máximo divisor comum entre os dois
        e = ZZ.random_element(phi)
        while gcd(e, phi) != 1:
            e = ZZ.random_element(phi)

        # Escolher d tal que d seja o inverso multiplicativo de e
        d = inverse_mod(e, phi)

        # Chaves
        self.private_key = (n, d)
        self.public_key  = (n, e)

    def encapsulation(self, peer_public_key, my_m):
        n, e = peer_public_key

        if (my_m == 0):
            # Encontrar m tal que 1 < m < n e depois encriptar
            m = ZZ.random_element(n)
        else:
            m = my_m
        m_encrypted = self.__encrypt(m, peer_public_key)

        # Obter chave
        m_in_bytes = int.to_bytes(int(m), int(m).bit_length(), "big")
        symetric_key = HKDF(
            algorithm=hashes.SHA256(),
            length=32,
            salt=self.salt,
            info=None,
        ).derive(m_in_bytes)

        # Podemos mandar sem ser em bytes?        
        return symetric_key, m_encrypted

    def decapsulation(self, m_encrypted):
        m = self.__decrypt(m_encrypted)
        m_in_bytes = int.to_bytes(int(m), int(m).bit_length(), "big")
        symetric_key = HKDF(
            algorithm=hashes.SHA256(),
            length=32,
            salt=self.salt,
            info=None,
        ).derive(m_in_bytes)

        return symetric_key

### Alínea (a)

In [168]:
salt = os.urandom(12)

bob_RSA   = KEM_RSA(salt)
alice_RSA = KEM_RSA(salt)

# Gerar chaves
bob_RSA.generate_keys(1024)
alice_RSA.generate_keys(1024)


symetric_key_bob, m_encrypted = bob_RSA.encapsulation(alice_RSA.public_key, 0)
symetric_key_alice = alice_RSA.decapsulation(m_encrypted)

print("Chave do lado do bob:   ", symetric_key_bob)
print("Chave do lado da alice: ", symetric_key_alice)

Chave do lado do bob:    b'\xe1]\xe6\xb0\x06}\xe2\x9e\xee\x99\xb4\xf0\x00\xf4\x8bW\xcd\xde\xa2\xd2\x81u\x83"\x9eE>\x92\xde\xb8n\x9e'
Chave do lado da alice:  b'\xe1]\xe6\xb0\x06}\xe2\x9e\xee\x99\xb4\xf0\x00\xf4\x8bW\xcd\xde\xa2\xd2\x81u\x83"\x9eE>\x92\xde\xb8n\x9e'


### Alínea (b)

In [173]:
class PKE:
    def __init__(self, kem):
        self.kem = kem
    

    # Metodos Privados
    def __hash_functio_h(self, message):
        digest = hashes.Hash(hashes.SHA256())
        digest.update(message)
        return digest.finalize()
    
    def __hash_functio_g(self, message):
        digest = hashes.Hash(hashes.BLAKE2s(32))
        digest.update(message)
        return digest.finalize()


    # Metodos Publicos
    def encrypt(self, message, peer_public_key):
        # r <- h(message)
        r = self.__hash_functio_h(message)
        # y <- message XOR g(r)
        y = bytes([a ^^ b for a, b in zip(message, self.__hash_functio_g(r))])
        # Concatenamos y e r
        new_r = y + r
        new_r_int = int.from_bytes(new_r, "big") 

        # Vamos agora usar o KEM construido na alinea anterior
        symetric_key, m_encrypted = self.kem.encapsulation(peer_public_key, new_r_int)
        
        # c = symetric_key XOR r
        c = bytes([a ^^ b for a, b in zip(symetric_key, r)])

        return y, m_encrypted, c
        
    def decrypt(self, y, m_encrypted, c):
        # Usamos o KEM para obter a chave
        symetric_key = self.kem.decapsulation(m_encrypted)
        
        # Repetimos o processo mas com r <- c XOR symetric_key
        r = bytes([a ^^ b for a, b in zip(c, symetric_key)])
        new_r = y + r
        new_r_int = int.from_bytes(new_r, "big") 

        # Verificamos pois f(public_key, new_r) = (symetric_key, m_encrypted)
        new_symetric_key, new_m_encrypted = self.kem.encapsulation(self.kem.public_key, new_r_int)
        if symetric_key != new_symetric_key:
            print("Not YO: symetric_key is diferent")
        else:
            if m_encrypted != new_m_encrypted:
                print("Not YO: m_encrypted is diferent")
            else:
                message = bytes([a ^^ b for a, b in zip(y, self.__hash_functio_g(r))])
                print("Yo the message is: ", message)

In [174]:
salt = os.urandom(12)

bob_RSA = KEM_RSA(salt)
bob_RSA.generate_keys(1024)

alice_RSA = KEM_RSA(salt)
alice_RSA.generate_keys(1024)


bob_PKE = PKE(bob_RSA)
y, m_encrypted, c = bob_PKE.encrypt(b"This message is YO", alice_RSA.public_key)

alice_PKE = PKE(alice_RSA)
alice_PKE.decrypt(y, m_encrypted, c)

Yo the message is:  b'This message is YO'
